In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from cmath import nan
import pywt
import torch
import pywt
from pandarallel import pandarallel
import tqdm
from functools import partial
from pathlib import Path
pandarallel.initialize(progress_bar=True)

/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
from numpy.lib.stride_tricks import as_strided

def sliding_window(data, win=29, s=1):
    itemsize  = data.itemsize
    N = len(data)
    sliding_data = as_strided(data, shape=((N - win) // s + 1, win), strides=(itemsize*s, itemsize))
    return sliding_data

In [4]:
def convert_canid_bits(cid):
    try:
        s = bin(int(str(cid), 16))[2:].zfill(29)
    except:
        s = bin(int(str(int(float(cid))), 16))[2:].zfill(29)
    bits = np.array(list(map(int, list(s))))
    return bits

In [5]:
label_to_num = {
    'Normal': 0,
    'Fuzzing': 1, 
    'Replay': 2
   }
def preprocess_per_row(sample):
   sample['bid']  = convert_canid_bits(sample['CAN_ID'])
   sample['label'] = label_to_num[sample['Label']]
   return sample

In [35]:
data_path = Path('../../../Data/LISA/Federated_Data/Labeled data/')
car_model = 'Tesla'
label = 'Replay'
file_id = 1
file_name = f'{car_model}_{label}_{file_id}.csv'
df = pd.read_csv(data_path / file_name)
df.CAN_ID = df.CAN_ID.astype(str)
# df = df.drop(['Time_Gap'], axis=1)
df

,Time_Ofset,Type,CAN_ID,Data_Length,Data_0,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Time_Gap,Scaled_CAN_ID,Label
0,188809.0,Rx,038D,7,41,00,00,00,00,40,46,NaN,0.1,50,Replay
1,188809.0,Rx,0105,8,00,00,00,00,40,06,00,4C,0.0,4,Replay
2,188809.0,Rx,011D,8,9E,00,00,00,00,00,80,00,0.0,10,Replay
3,188809.0,Rx,0129,8,1B,2B,38,62,1C,20,FF,3F,0.0,11,Replay
4,188810.0,Rx,0145,8,5B,40,00,2F,A4,00,02,00,1.0,14,Replay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77995,372966.8,Rx,0118,8,95,0B,85,18,54,80,00,00,0.0,7,Replay
77996,372966.9,Rx,0206,8,00,00,00,00,00,00,60,68,0.1,23,Replay
77997,372966.9,Rx,0148,8,B7,06,00,40,12,00,16,00,0.0,15,Replay
77998,372968.0,Rx,0257,8,51,F6,3D,27,9E,00,00,00,1.1,29,Replay


In [36]:
df = df.parallel_apply(preprocess_per_row, axis=1)

In [37]:
window_size = 29 
strided = 13
can_id_sequences = sliding_window(df.bid.to_numpy(), win=window_size, s=strided)
can_id_sequences = np.array([np.stack(x) for x in can_id_sequences]).astype('int8')
label_sequences = sliding_window(df.label.to_numpy(), win=window_size, s=strided)
labels = np.any(label_sequences, axis=1).astype('int8')
labels[np.where(labels > 0)] = label_sequences[0][0]

In [38]:
output_path = Path('../../../Data/LISA/Federated_Data/Preprocessed_Data/')
output_path.mkdir(parents=True, exist_ok=True)
np.savez_compressed(output_path / file_name[:-4], X=can_id_sequences, y=labels)

## Load data

In [22]:
data_path = Path('../../../Data/LISA/Federated_Data/Preprocessed_Data/')
car_model = 'Kia'
label = 'Fuzz'
file_id = 1
file_name = f'{car_model}_{label}_{file_id}.npz'

In [23]:
data = np.load(data_path / file_name)

In [24]:
X, y = data['X'], data['y']
print(X.shape)
print(y.shape)

(35868, 29, 29)
(35868,)
